### 1. Backbone

In [17]:
from mmdet.models import build_backbone
from mmcv import Config

cfg = Config.fromfile('configs/model_modules.py')
backbone = build_backbone(cfg.backbone)

In [73]:
import torch

input = torch.randn(4, 3, 512, 512)
output_backbone = backbone(input)

print('input type:', type(input)) # = Tensor
print('input shape', input.shape) # shape of feature maps from each stage
print()
print('output type:', type(output_backbone)) # = tuple
print('output length:', len(output_backbone)) # = number of stages in backbone
print('output item type:', type(output_backbone[0])) # = Tensor
print('output item shapes', tuple(tuple(t.shape) for t in output_backbone)) # shape of feature maps from each stage

input type: <class 'torch.Tensor'>
input shape torch.Size([4, 3, 512, 512])

output type: <class 'tuple'>
output length: 4
output item type: <class 'torch.Tensor'>
output item shapes ((4, 256, 128, 128), (4, 512, 64, 64), (4, 1024, 32, 32), (4, 2048, 16, 16))


### 2. Neck

In [74]:
from mmdet.models import build_neck

neck = build_neck(cfg.neck)
output_neck = neck(output_backbone)

print('output type:', type(output_neck)) # = tuple
print('output length:', len(output_neck)) # = number of stages in backbone
print('output item type:', type(output_neck[0])) # = Tensor
print('output item shapes', tuple(tuple(t.shape) for t in output_neck)) # shape of feature maps from each stage

output type: <class 'tuple'>
output length: 5
output item type: <class 'torch.Tensor'>
output item shapes ((4, 256, 128, 128), (4, 256, 64, 64), (4, 256, 32, 32), (4, 256, 16, 16), (4, 256, 8, 8))


In [75]:
[(512//n, 512//n) for n in [4, 8, 16, 32, 64]]

[(128, 128), (64, 64), (32, 32), (16, 16), (8, 8)]

In [71]:
from mmdet.core import AnchorGenerator
anchor_gen = AnchorGenerator(scales=[8],
            ratios=[1.0],
            strides=[4, 8, 16, 32, 64])

In [76]:
anchor_gen.grid_anchors([(128, 128), (64, 64), (32, 32), (16, 16), (8, 8)], device='cpu')

[tensor([[-16., -16.,  16.,  16.],
         [-12., -16.,  20.,  16.],
         [ -8., -16.,  24.,  16.],
         ...,
         [484., 492., 516., 524.],
         [488., 492., 520., 524.],
         [492., 492., 524., 524.]]),
 tensor([[-32., -32.,  32.,  32.],
         [-24., -32.,  40.,  32.],
         [-16., -32.,  48.,  32.],
         ...,
         [456., 472., 520., 536.],
         [464., 472., 528., 536.],
         [472., 472., 536., 536.]]),
 tensor([[-64., -64.,  64.,  64.],
         [-48., -64.,  80.,  64.],
         [-32., -64.,  96.,  64.],
         ...,
         [400., 432., 528., 560.],
         [416., 432., 544., 560.],
         [432., 432., 560., 560.]]),
 tensor([[-128., -128.,  128.,  128.],
         [ -96., -128.,  160.,  128.],
         [ -64., -128.,  192.,  128.],
         ...,
         [ 288.,  352.,  544.,  608.],
         [ 320.,  352.,  576.,  608.],
         [ 352.,  352.,  608.,  608.]]),
 tensor([[-256., -256.,  256.,  256.],
         [-192., -256.,  320.,  2

In [36]:
anchor_gen.grid_priors([(2, 2), (10,10)], device='cpu')

AssertionError: 

In [83]:
cfg = Config.fromfile('/opt/ml/detection/mmdetection/my_configs/faster_rcnn_base.py')
print(cfg.pretty_text)

Exception ignored in: <function _TemporaryFileCloser.__del__ at 0x7fe07e412a60>
Traceback (most recent call last):
  File "/opt/conda/envs/mmdetection/lib/python3.8/tempfile.py", line 441, in __del__
    self.close()
  File "/opt/conda/envs/mmdetection/lib/python3.8/tempfile.py", line 437, in close
    unlink(self.name)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpfva7_j9t/tmp84yv24s7.py'


model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [ ]:
        >>> all_anchors = self.grid_anchors([(2, 2)], device='cpu')
        >>> print(all_anchors)
        [tensor([[-4.5000, -4.5000,  4.5000,  4.5000],
                [11.5000, -4.5000, 20.5000,  4.5000],
                [-4.5000, 11.5000,  4.5000, 20.5000],
                [11.5000, 11.5000, 20.5000, 20.5000]])]
        >>> self = AnchorGenerator([16, 32], [1.], [1.], [9, 18])
        >>> all_anchors = self.grid_anchors([(2, 2), (1, 1)], device='cpu')
        >>> print(all_anchors)
        [tensor([[-4.5000, -4.5000,  4.5000,  4.5000],
                [11.5000, -4.5000, 20.5000,  4.5000],
                [-4.5000, 11.5000,  4.5000, 20.5000],
                [11.5000, 11.5000, 20.5000, 20.5000]]), \
        tensor([[-9., -9., 9., 9.]])]